# Naive Bayes Classifier

## Import data

In [139]:
import pandas as pd 
import numpy as np
import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [160]:
results = pd.read_csv('./cleaned_data/cleaned_asylum_seekers_added.csv')
results

/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Year,Country / territory of asylum/residence,Origin,RSD procedure type / level,decisions_recognized,decisions_other,Rejected,Otherwise_closed,Total decisions,Successful,...,Origin_latitude,Origin_longitude,origin_to_target_dist,HDI,Unemployment rate,acceptance_rate,accepted/rejected,Encoded procedure type,Encoded Target Country,Encoded Origin
0,2001,South Africa,Angola,G / AR,0.0,0.0,0.0,1.0,0.0,0.0,...,-14.270972,-170.132217,11214.770329,0.61,30.896,0.000000,0,1,135,3
1,2001,South Africa,Angola,G / FI,860.0,0.0,36.0,68.0,964.0,860.0,...,-14.270972,-170.132217,11214.770329,0.61,30.896,0.892116,3,6,135,3
2,2001,South Africa,Albania,G / AR,0.0,0.0,0.0,1.0,0.0,0.0,...,41.153332,20.168331,11919.880369,0.61,30.896,0.000000,0,1,135,1
3,2001,South Africa,Burundi,G / AR,0.0,0.0,0.0,1.0,0.0,0.0,...,12.238333,-1.561593,8114.662998,0.61,30.896,0.000000,0,1,135,25
4,2001,South Africa,Burundi,G / FI,808.0,0.0,234.0,7.0,1049.0,808.0,...,12.238333,-1.561593,8114.662998,0.61,30.896,0.770257,3,6,135,25
5,2001,South Africa,Benin,G / AR,0.0,0.0,0.0,1.0,0.0,0.0,...,17.189877,-88.497650,9356.444643,0.61,30.896,0.000000,0,1,135,18
6,2001,South Africa,Benin,G / FI,0.0,0.0,75.0,5.0,80.0,0.0,...,17.189877,-88.497650,9356.444643,0.61,30.896,0.000000,0,6,135,18
7,2001,South Africa,Burkina Faso,G / AR,0.0,0.0,0.0,1.0,0.0,0.0,...,42.733883,25.485830,12301.788683,0.61,30.896,0.000000,0,1,135,24
8,2001,South Africa,Bangladesh,G / AR,0.0,0.0,0.0,1.0,0.0,0.0,...,25.930414,50.637772,12156.012618,0.61,30.896,0.000000,0,1,135,13
9,2001,South Africa,Bangladesh,G / FI,0.0,0.0,268.0,30.0,298.0,0.0,...,25.930414,50.637772,12156.012618,0.61,30.896,0.000000,0,6,135,13


In [161]:
# Top 10 countrys refugee originated from 
# 1. Syrian Arab Rep, 2. Afghanistan, 3. Iraq, 4. Serbia and Kosova, 5. China
# 6. Russian Federation, 7. Iran (Islamic Rep of), 8. Pakistan, 9. Somalia, 10. Nigeria

In [162]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109330 entries, 0 to 109329
Data columns (total 26 columns):
Year                                       109330 non-null int64
Country / territory of asylum/residence    109330 non-null object
Origin                                     109330 non-null object
RSD procedure type / level                 109330 non-null object
decisions_recognized                       109330 non-null float64
decisions_other                            109330 non-null float64
Rejected                                   109330 non-null float64
Otherwise_closed                           109330 non-null float64
Total decisions                            109330 non-null float64
Successful                                 109330 non-null float64
Unsuccessful                               109330 non-null float64
Target_country_GDP_per_capita              109330 non-null float64
Origin_country_GDP_per_capita              109330 non-null float64
GDP_difference         

## Multinomial Naive Bayes

In [183]:
# Declaring label
label = results['accepted/rejected']

# Combining features into a single variable
results = results[results['HDI'] != '..']

# Convert to float type
results['HDI'] = results['HDI'].astype('float64')
results = results.dropna()

# results['GDP_difference'] = results['GDP_difference'].abs() 

col_names = ['Encoded procedure type', 'HDI', 'Unemployment rate', 'origin_to_target_dist']
# features = results.iloc[:,-3:]
features = results[col_names]

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.20)

nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_class = nb.predict(X_test)
print("y_pred: ", y_pred_class)

y_pred:  [3 0 0 ... 0 0 0]


/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [184]:
print("accuracy: ", metrics.accuracy_score(y_test, y_pred_class))
print(confusion_matrix(y_test, y_pred_class))
print()
print(classification_report(y_test, y_pred_class))

accuracy:  0.735529217199559
[[15853   308     0  1674]
 [ 2087    59     0   192]
 [  835    16     0    97]
 [  539     9     0    99]]

              precision    recall  f1-score   support

           0       0.82      0.89      0.85     17835
           1       0.15      0.03      0.04      2338
           2       0.00      0.00      0.00       948
           3       0.05      0.15      0.07       647

   micro avg       0.74      0.74      0.74     21768
   macro avg       0.25      0.27      0.24     21768
weighted avg       0.69      0.74      0.71     21768



/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## GaussianNB

In [145]:
# Declaring label
label = results['accepted/rejected']

# Combining features into a single variable
results = results[results['HDI'] != '..']

# Convert to float type
results['HDI'] = results['HDI'].astype('float64')
results = results.dropna()

# Combining features into a single variable
col_names = ['Encoded procedure type', 'Encoded Target Country', 'Encoded Origin', 'HDI', 'Unemployment rate',
            'origin_to_target_dist']
# features = results.iloc[:,-3:]
features = results[col_names]

# Train the model using the training sets
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.20, random_state=42)

# Creating a Gaussian Classifier
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("y_pred", y_pred)
print("accuracy: ", metrics.accuracy_score(y_test, y_pred))

y_pred [0 0 0 ... 0 0 0]
accuracy:  0.818632855567806


/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [146]:
print(confusion_matrix(y_test, y_pred))
print()
print(classification_report(y_test, y_pred))

[[17820     0     0     0]
 [ 2250     0     0     0]
 [  992     0     0     0]
 [  706     0     0     0]]

              precision    recall  f1-score   support

           0       0.82      1.00      0.90     17820
           1       0.00      0.00      0.00      2250
           2       0.00      0.00      0.00       992
           3       0.00      0.00      0.00       706

   micro avg       0.82      0.82      0.82     21768
   macro avg       0.20      0.25      0.23     21768
weighted avg       0.67      0.82      0.74     21768



/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## BernoulliNB